In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_recall_curve, roc_auc_score

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
df = pd.read_csv("./train_case2.csv", delimiter=";", header=0, index_col=0)

In [4]:
df.head(3)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, "age":"active"], df["cardio"], random_state=42, test_size=0.3)

In [6]:
# сделаем транформер для колонок, категориальные трансформируем с помощью OneHot, непрерывные с помощью StandartScaler

In [7]:
col_trans = make_column_transformer(
                                    (OneHotEncoder(), ["gender", "cholesterol", "gluc", "smoke", "alco"]),
                                    (StandardScaler(), ["age", "height", "weight", "ap_hi", "ap_lo"])
                                    )

In [8]:
# делаем пайплайн для логрес модели, обучаем ее делаем 

In [9]:
log_res = LogisticRegression(C=0.7, n_jobs=-1, random_state=42)

In [10]:
logres_pipe = make_pipeline(col_trans, log_res)

In [11]:
cv_lr_scores = cross_val_score(logres_pipe, X_train, y_train, cv=10, scoring="roc_auc")

print(f"Cross-validation score for LogRess is {np.mean(cv_lr_scores):.5f} "
      f"mean with {np.std(cv_lr_scores):.5f} standart deviation")

Cross-validation score for LogRess is 0.77946 mean with 0.00564 standart deviation


In [12]:
logres_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['gender', 'cholesterol',
                                                   'gluc', 'smoke', 'alco']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'height', 'weight',
                                                   'ap_hi', 'ap_lo'])])),
                ('logisticregression',
                 LogisticRegression(C=0.7, n_jobs=-1, random_state=42))])

In [13]:
lr_pred = logres_pipe.predict_proba(X_test)[:, 1]

In [14]:
# найдем лучший порог для precision&recall и оценим roc_auc

In [15]:
def best_threshold(y_test, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_test, lr_pred)
    fscore = 2*(precision * recall) / (2*precision + recall)
    best = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, y_pred)
    return thresholds[best], precision[best], recall[best], fscore[best], roc_auc

In [16]:
threshold_lr, precision_lr, recall_lr, f_score_lr, roc_auc_lr = best_threshold(y_test, lr_pred)
print(f"Precision={precision_lr:.5f} recall={recall_lr:.5f}, fscore={f_score_lr:.5f}, threshold={threshold_lr:.5f}, "
      f"roc_auc={roc_auc_lr:.5f}")

Precision=0.59731 recall=0.91821, fscore=0.51917, threshold=0.33337, roc_auc=0.78382


In [17]:
# если смотреть на roc_auc то может показаться что модель дает хорошую прогнозную силу. и это действительно так,
# пока мы не посмотрим на оценку Precision. recall дает достаточно хороший результат, что говорит о том, что модель выбирает
# почти все классы с меткой 1, но precision на уровне 60% говорит о том, что модель достаточно много классов оценивает как
# ложноположительные. Возможно с точки зрения оценки кардиозаболеваний такой подход оправдан, лучше отправить здорового
# человека на дополнительные исследования, чем потом лечить хронического. Т.е. в данном случае модель
# минимизирует ошибку первого рода

In [18]:
# сделаем такой же пайплайн для RandomForest

In [19]:
rand_forest = RandomForestClassifier(n_estimators=150, n_jobs=-1, max_depth=7)

In [20]:
rand_forest_pipe = make_pipeline(col_trans, rand_forest)

In [21]:
cv_rf_scores = cross_val_score(rand_forest_pipe, X_train, y_train, cv=10, scoring="roc_auc")

print(f"Cross-validation score for RandomForest is {np.mean(cv_rf_scores):.5f} "
      f"mean with {np.std(cv_rf_scores):.5f} standart deviation")

Cross-validation score for RandomForest is 0.79821 mean with 0.00512 standart deviation


In [22]:
# как видно на этапе кросс-валидации оценка случайного леса чуть лучше чем логистической регрессии. Чуть выше оценка плюс 
# чуть меньше ее разброс, однако оценка идет для roc_auс, посмотрим на остальные метрики

In [23]:
rand_forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['gender', 'cholesterol',
                                                   'gluc', 'smoke', 'alco']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'height', 'weight',
                                                   'ap_hi', 'ap_lo'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=7, n_estimators=150,
                                        n_jobs=-1))])

In [24]:
rf_pred = rand_forest_pipe.predict_proba(X_test)[:, 1]

In [25]:
threshold_rf, precision_rf, recall_rf, f_score_rf, roc_auc_rf = best_threshold(y_test, rf_pred)
print(f"Precision={precision_rf:.5f} recall={recall_rf:.5f}, fscore={f_score_rf:.5f}, threshold={threshold_rf:.5f}, "
      f"roc_auc={roc_auc_rf:.5f}")

Precision=0.59731 recall=0.91821, fscore=0.51917, threshold=0.33337, roc_auc=0.80074


In [26]:
# сделаем пайплайн для градиентного бустинга

In [27]:
grad_boost = GradientBoostingClassifier(n_estimators=150, max_depth=7, random_state=42)

In [28]:
grad_boost_pipe = make_pipeline(col_trans, grad_boost)

In [29]:
cv_gb_scores = cross_val_score(grad_boost_pipe, X_train, y_train, cv=10, scoring="roc_auc")

print(f"Cross-validation score for GradientBoost is {np.mean(cv_gb_scores):.5f} "
      f"mean with {np.std(cv_gb_scores):.5f} standart deviation")

Cross-validation score for GradientBoost is 0.79677 mean with 0.00438 standart deviation


In [30]:
grad_boost_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['gender', 'cholesterol',
                                                   'gluc', 'smoke', 'alco']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'height', 'weight',
                                                   'ap_hi', 'ap_lo'])])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=7, n_estimators=150,
                                            random_state=42))])

In [31]:
gb_pred = grad_boost_pipe.predict_proba(X_test)[:, 1]

In [35]:
threshold_gb, precision_gb, recall_gb, f_score_gb, roc_auc_gb = best_threshold(y_test, gb_pred)
print(f"Precision={precision_gb:.5f} recall={recall_gb:.5f}, fscore={f_score_gb:.5f}, threshold={threshold_gb:.5f}, "
      f"roc_auc={roc_auc_gb:.5f}")

Precision=0.59731 recall=0.91821, fscore=0.51917, threshold=0.33337, roc_auc=0.79923


In [36]:
metrics_df = pd.DataFrame({"Precision": [precision_lr, precision_rf, precision_gb],
                           "Recall": [recall_lr, recall_rf, recall_gb],
                           "F1-score": [f_score_lr, f_score_rf, f_score_gb],
                           "threshold": [threshold_lr, threshold_rf, threshold_gb],
                           "roc_auc": [roc_auc_lr, roc_auc_rf, roc_auc_gb]},
                          index=["LogisticRegression", "RandomForest", "GradientBoosting"])

In [37]:
metrics_df

,Precision,Recall,F1-score,threshold,roc_auc
LogisticRegression,0.597309,0.918209,0.519166,0.333367,0.783817
RandomForest,0.597309,0.918209,0.519166,0.333367,0.800745
GradientBoosting,0.597309,0.918209,0.519166,0.333367,0.799230


In [41]:
# эмммм немного странные метрики получились. Но при этом уровень roc_auc все же разный. Как мне кажется это связано с масштабом 
# предсказаний, возможно модели делают примерно одинаковые TP и TN предсказания, но различающиеся на несколько единиц 
# и из-за этого даже с учетом 6 знаков после запятой оценки precision и recall не улавливают данные колебания
# roc_auc ведет себя немного по другому и указывают на определенные различия в моделях

(опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90  
вторая помечает 1000 объектов как класс 1, но TP такой же - 90  
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

========================================

Возможно стоит промоделировать сам процесс расчета метрик, хотябы на словах
При использовании precision и recall вторая метрика дает нам оценку предсказательной силы, а первая не дает записать все объекты в целевой класс.   
В случае с первой моделью Precision=90/100=0.9, recall=90/100=0.9  
В случае со второй моделью Precision=90/1000=0.09, recall=90/100=0.9
И если выбирать на основании чисто этих метрик, предпочтительней будет первая модель, потому что она меньше ошибается в случае ложноположительных оценок. На графике линия второй модели будет лежать ниже линии первой модели.  

В случае с roc_auc будет следующая картинка
Перед построением кривой выданные ответы будут проранжированы по убыванию, соответсвенно если представить таблицу, то вверху  у нас будет положительные ответы (100 для первой модели, 1000 для второй), а далее отрицательные ответы, при этом сама сетка построения кривой будет разбита на 100 вертикальных линий (правильно положительные ответы) и 99900 горизонтальных (правильно отрицательные) по которым и будет проходить кривая. 
При построении кривой по первой модели скорее всего линия резко пойдет вверх и дойдет или почти дойдет до 0.9 (поскольку мы ранжируем первые 100 положительных ответов), а дальше будет двигаться вправо.  А в случае со второй моделью, она также сначала пойдет вверх до 0.9 и далее так же пойдет вправо, потому что остальные 910 объектов истино отрицательные....возможно эти две линии не будут похожи друг на друга, но будут очень близки и соответсвенно auc будет различаться не сильно между ними и выбор оптимальной модели будет затруднителен. Поэтому с точки зрения дисбаланса классов лучшим выбором является precision_recall для оценки моделей